<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/09_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree-based methodsによる住宅価格の予測

* 前に使ったCalifornia housing datasetを使う。

  * scikit-learnからロードできるバージョンは、前処理が済んだキレイなデータなので、ここでは使わない。

* 今回は、以下の解説を参考にします。
  * https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html
* 特に、`sklearn.pipeline.make_pipeline`を用いることで、コードを簡素化します。

In [ ]:
import numpy as np
from scipy import stats, special
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

%config InlineBackend.figure_format = "retina"

## データセットを取得

In [ ]:
!wget https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz
!tar zxvf housing.tgz

In [ ]:
df = pd.read_csv("housing.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df_onehot = pd.get_dummies(df, dtype=int)

In [ ]:
df_onehot.head()

In [ ]:
df_onehot.info()

* 説明変数と目的変数を分ける



In [ ]:
X = df_onehot.drop('median_house_value', axis=1)
y = df_onehot["median_house_value"].copy()

## 評価実験のための準備

* 今回は交差検証をおこなうので、test setだけを切り分けておく。
  * test setの切り分け方は`06_linear_regression_1.ipynb`と同じ。
  * 交差検証には`sklearn.model_selection.cross_val_score`を使う。


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [ ]:
X_train.info()

In [ ]:
X_test.info()

## 前処理

* 欠測箇所がある。
  * 今回は`SimpleImputer`を使ってtraining setの中央値で埋めることにする。

## 決定木（今日のベースライン）

* 木の深さ（領域を何分割するか）をチューニングする。

* `cross_val_score`で使える評価尺度一覧をチェックする。
  * いずれも、「値が大きいほど良い」という評価尺度になっている。
  * 今回はRMSEで評価したいが、これは「値が小さいほど良い」という評価尺度である。
  * そこで、一覧の中にある`neg_root_mean_squared_error`に、マイナスをつけたものを、使う。
  * これで`06_linear_regression_1.ipynb`の時の結果と比較できるようになる。



In [ ]:
sorted(metrics.SCORERS.keys())

* 交差検証を、欠損値を埋めつつ実行するために、ヘルパ関数を定義しておく。

In [ ]:
def get_scores(imputer, regressor, X_missing, y_missing, cv=10):
  estimator = make_pipeline(imputer, regressor)
  return cross_val_score(
      estimator,
      X_missing, y_missing,
      scoring="neg_root_mean_squared_error",
      cv=cv,
  )

* 10-fold cross validationを実施
  * 木の深さを変える。
  * 評価尺度はRMSE。



In [ ]:
best_depth = 0
best_rmse = 0.0
for i in range(3, 16):
  reg = DecisionTreeRegressor(max_depth=i, random_state=123)
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  temp_rmse = - scores.mean()
  if best_depth == 0 or best_rmse > temp_rmse:
    best_depth = i
    best_rmse = temp_rmse
  print(f"depth {i}: {temp_rmse:.2f}")
print('-'*64)
print(f"best depth {best_depth}: {best_rmse:.2f}")

* 最適な木の深さを使ってtraining set全体で学習をやり直し、test setで評価。
  * test setでimputationするときは、training setから得た統計量を使うこと!
  * test setの中で平均値や中央値などの統計量を求めるのはNGです。



In [ ]:
reg = DecisionTreeRegressor(max_depth=best_depth, random_state=123)
imp = SimpleImputer(missing_values=np.nan, strategy="median")
reg.fit(imp.fit_transform(X_train), y_train)
y_test_pred = reg.predict(imp.transform(X_test))
rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"test RMSE: {rmse:.2f}")

* これが今回のベースラインです。
  * この予測性能を改善することを試みてください。

# 課題

* RMSEによって評価される予測性能を、良くして下さい
* test setとそれ以外の部分の分割は、変えないでください
  * test set以外の部分をどう使うかは、自由です
  * 交差検証の方法は何でもよいです。
* 決定木ベースの手法やgradient boosting系の手法なら、何を使ってもいいです。
* test setでのRMSEによる評価は、最後に一回おこなうだけです



---



# XGBoost



In [ ]:
import xgboost as xgb

for i in range(6, 14):
  reg = xgb.XGBRegressor(objective ="reg:squarederror", max_depth=i, random_state=123)
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  print(f"depth {i}: {- scores.mean():.2f}")

* 最適な木の深さを使ってtraining set全体で学習をやり直し、test setで評価。



In [ ]:
# 各自、実践してください。

# CatBoost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

for i in range(6, 14):
  reg = CatBoostRegressor(iterations=200, depth=i, random_seed=123, logging_level="Silent")
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  print(f"depth {i}: {- scores.mean():.2f}")

* 最適な木の深さを使ってtraining set全体で学習をやり直し、test setで評価。

In [ ]:
# 各自、実践してください。

* 他に、決定木に類するモデル、gradient boosting系のモデルを見つけて、色々試してみてください。
  * それぞれのモデルのハイパーパラメータをチューニングする。
  * チューニングしたハイパーパラメータを使って、test setでの評価値を得る。
  * どの手法が、test setで最も良い評価値を与えたか、比較する。